In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 
%matplotlib inline 
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns
from keras.applications import xception
from keras.preprocessing import image
from mpl_toolkits.axes_grid1 import ImageGrid
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix
from tqdm import tqdm
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all" 
import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
start = dt.datetime.now()

In [ ]:
!ls ../input/keras-pretrained-models/

In [ ]:
#이미 학습된 케라스 모델(Keras Pretrained Model) 중에서 선택하여 적용할 것.
# copy the pretrained models to the cache directroy
cache_dir = os.path.expanduser(os.path.join('~', '.keras')) # expanduser: ~를 사용자의 홈 디렉토리 절대경로로 바꿈

if not os.path.exists(cache_dir):
    os.makedirs(cache_dir)
models_dir = os.path.join(cache_dir, 'models')
if not os.path.exists(models_dir):
    os.makedirs(models_dir)
    

In [ ]:
!cp ../input/keras-pretrained-models/xception* ~/.keras/models/

In [ ]:
!ls ~/.keras/models

In [ ]:
CATEGORIES = ['Black-grass', 'Charlock', 'Cleavers', 'Common Chickweed', 'Common wheat', 'Fat Hen', 'Loose Silky-bent',
              'Maize', 'Scentless Mayweed', 'Shepherds Purse', 'Small-flowered Cranesbill', 'Sugar beet']
NUM_CATEGORIES = len(CATEGORIES)

In [ ]:
SAMPLE_PER_CATEGORY = 200
SEED = 1987
data_dir = '../input/plant-seedlings-classification/'
train_dir = os.path.join(data_dir, 'train')
test_dir = os.path.join(data_dir, 'test')
sample_submission = pd.read_csv(os.path.join(data_dir, 'sample_submission.csv'))

In [ ]:
sample_submission.head(2)

train data에 각 식물이 몇개씩 있는지 확인.

In [ ]:
for category in CATEGORIES:
    print('{} {} images'.format(category, len(os.listdir(os.path.join(train_dir, category)))))


In [ ]:
train = []
for category_id, category in enumerate(CATEGORIES):
    for file in os.listdir(os.path.join(train_dir, category)):
        train.append(['train/{}/{}'.format(category, file), category_id, category])
train = pd.DataFrame(train, columns=['file', 'category_id', 'category'])
train.head(2)
train.shape

pd.concat() : 동일한 형태의 Dataframe을 합침.
SAMPLE_PER_CATEGORY = 200 이므로, 카테고리가 c인 element에서 각각 200개씩 빼서 샘플을 만든다.

train.sample(frac=1): shuffle할때쓰이는 함수. frac = fraction of rows to return in the random sample

np.arange(): range()함수의 nummpy버전. 0~(n-1)


In [ ]:
train = pd.concat([train[train['category'] == c][:SAMPLE_PER_CATEGORY] for c in CATEGORIES])
train = train.sample(frac=1)
train.index = np.arange(len(train))
train.head(2)
train.shape

In [ ]:
test = []
for file in os.listdir(test_dir):
    test.append(['test/{}'.format(file), file])

test = pd.DataFrame(test, columns = ['filepath', 'file'])
test.head(2)
test.shape

In [ ]:
def read_img(filepath, size):
    img = image.load_img(os.path.join(data_dir, filepath), target_size = size)
    img = image.img_to_array(img)
    return img

plt.figure : new figure(그림이나 도형)을 생성.
ImageGrid에서 111은 축의 위치를 나타내는 숫자.

In [ ]:
fig = plt.figure(1, figsize = (NUM_CATEGORIES, NUM_CATEGORIES))
grid = ImageGrid(fig, 111, nrows_ncols=(NUM_CATEGORIES, NUM_CATEGORIES), axes_pad=0.05)
i = 0
for category_id, category in enumerate(CATEGORIES):
    for filepath in train[train['category'] == category]['file'].values[:NUM_CATEGORIES]:
        ax = grid[i]
        img = read_img(filepath, (224,224))
        ax.imshow(img / 255.)
        ax.axis('off')
        if i % NUM_CATEGORIES == NUM_CATEGORIES - 1:
            ax.text(250, 112,filepath.split('/')[1], verticalalignment='center')
        i += 1

plt.show();

validation set 만들기

np.random.random(n)은 0이상 1미만의 실수를 n의 사이즈만큼 리턴함.

train_idx는 각 원소들이 0.8보다 작은지 보고 boolean을 리턴함.

train.loc[train_idx,'category_id'].values : train_idx에서 참인 frame만 뽑아오는데 이때, 열이 'category_id'인 것들의 values만 뽑아오게 됨.



In [ ]:
np.random.seed(seed=SEED)
rnd = np.random.random(len(train))
train_idx = rnd < 0.8
valid_idx = rnd >= 0.8
ytr = train.loc[train_idx, 'category_id'].values
yv = train.loc[valid_idx, 'category_id'].values
len(ytr), len(yv)


tqdm : for문에서의 진행상태를 보여줌.

XCeption model
https://datascienceschool.net/view-notebook/0faaf59e0fcd455f92c1b9a1107958c4/

Depth-wise-separable convolution: 각 채널별로 conv하고, 그 결과에 1x1 conv 연산.
각 채널별로 feature map을 하나씩만들고(conv), feature map 의 수를 조정(1x1).

비선형 activation 함수가 없음.

ResNet의 skip connection도 적용됨. maxpool 다음에 위치해 있어서 이전입력에 대해 크기를 줄이는 연산을 함. 1x1 convolution에서는 stride = 2로 함.


expand_dims():axis=0이면 제일 앞에있는 차원이 추가되어 확장된다.

In [ ]:
INPUT_SIZE = 299
POOLING = 'avg'
x_train = np.zeros((len(train), INPUT_SIZE, INPUT_SIZE, 3), dtype='float32')
for i, file in tqdm(enumerate(train['file'])):
    img = read_img(file, (INPUT_SIZE, INPUT_SIZE))
    x = xception.preprocess_input(np.expand_dims(img.copy(),axis=0))
    x_train[i] = x
print('Train Images Shape : {} size: {:,}'.format(x_train.shape, x_train.size))

bottleneck feature
CNN의 앞쪽 레이어에서 특징추출하고, 뒤쪽레이어에서 그 특징을 가지고 이미지를 분류.

앞쪽 레이어를 통과시켜 추출된 특징 = bottleneck feature

{:,} : 서식지정자. 숫자 사이에 ,가 들어갈것.

verbose는 학습 중 출력되는 문구 설정하는 것.
include_top 은 마지막 layer를 선택할것인지 말지를 결정.

In [ ]:
Xtr = x_train[train_idx]
Xv = x_train[valid_idx]
print((Xtr.shape, Xv.shape, ytr.shape, yv.shape))
xception_bottleneck = xception.Xception(weights='imagenet', include_top=False, pooling=POOLING)
train_x_bf = xception_bottleneck.predict(Xtr, batch_size=32, verbose=1)
valid_x_bf = xception_bottleneck.predict(Xv, batch_size=32, verbose=1)
print('Xception train bottleneck features shape: {} size: {:,}'.format(train_x_bf.shape, train_x_bf.size))
print('Xception valid bottleneck features shape: {} size: {:,}'.format(valid_x_bf.shape, valid_x_bf.size))

multinomial : 범주가 3개이상이고 명목형(nomial)일때.

multi_class = {'ovr', 'multinomial', 'auto'} 디폴트는 ovr(이진형}. auto는 자동
solver : optimization알고리즘 선택. lbfgs는 L2

특징추출로 bottleneck을 쓰고, 분류로는 logistic regression을 씀.

logreg.fit() -> 회귀분석 적용


In [ ]:
logreg = LogisticRegression(multi_class='multinomial', solver='lbfgs', random_state=SEED)
logreg.fit(train_x_bf, ytr)
valid_probs = logreg.predict_proba(valid_x_bf)
valid_preds = logreg.predict(valid_x_bf)


In [ ]:
print('Validation Xception Accuracy {}'.format(accuracy_score(yv, valid_preds)))

Confusion Matrix: 성능지표를 나타낸 행렬(생략)

In [ ]:
cnf_matrix = confusion_matrix(yv, valid_preds)
abbreviation = ['BG', 'Ch', 'Cl', 'CC', 'CW', 'FH', 'LSB', 'M', 'SM', 'SP', 'SFC', 'SB']
pd.DataFrame({'class': CATEGORIES, 'abbreviation': abbreviation})
fig, ax = plt.subplots(1)
ax = sns.heatmap(cnf_matrix, ax=ax, cmap=plt.cm.Greens, annot=True)
ax.set_xticklabels(abbreviation)
ax.set_yticklabels(abbreviation)
plt.title('Confusion Matrix')
plt.ylabel('True class')
plt.xlabel('Predicted class')
fig.savefig('Confusion matrix.png', dpi=300)
plt.show();

TEST

In [ ]:
x_test = np.zeros((len(test), INPUT_SIZE, INPUT_SIZE, 3), dtype='float32')
for i, filepath in tqdm(enumerate(test['filepath'])):
    img = read_img(filepath, (INPUT_SIZE, INPUT_SIZE))
    x = xception.preprocess_input(np.expand_dims(img.copy(), axis=0))
    x_test[i] = x
print('test Images shape: {} size: {:,}'.format(x_test.shape, x_test.size))

In [ ]:
test_x_bf = xception_bottleneck.predict(x_test, batch_size=32, verbose=1)
print('Xception test bottleneck features shape: {} size: {:,}'.format(test_x_bf.shape, test_x_bf.size))
test_preds = logreg.predict(test_x_bf)

In [ ]:
test['category_id'] = test_preds
test['species'] = [CATEGORIES[c] for c in test_preds]
test[['file', 'species']].to_csv('submission.csv', index=False)

In [ ]:
end = dt.datetime.now()
print('Total time {} s.'.format((end - start).seconds))
print('We almost used the one hour time limit.')